In [ ]:
# %%capture suppress warnings
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Import required libraries
from unsloth import FastLanguageModel
from datasets import load_dataset
import torch

# Set parameters for LLaMA-3.1 8B finetuning on a coding task
max_seq_length = 1024  # Optimized for memory and performance
dtype = torch.float16  # Efficient for free T4 instance
load_in_4bit = True  # Reduced memory usage with 4bit quantization

# Load LLaMA-3.1 8B model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# Add LoRA adapters to finetune only parts of the model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Set LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Focus on attention layers
    lora_alpha=16,
    lora_dropout=0,  # Optimized dropout rate
    bias="none",  # Remove bias to optimize memory usage
    use_gradient_checkpointing=True  # Reduce memory further for long sequences
)

# Dataset for coding-related tasks: Use Alpaca or other coding datasets
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

# Function to format prompts
alpaca_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # End of sequence token

def formatting_prompts_func(examples):
    """
    Function to format the Alpaca dataset to match the model's required input.
    This formats the instruction, input, and response into a prompt format for fine-tuning.
    """
    texts = [alpaca_prompt.format(inst, inp, out) + EOS_TOKEN for inst, inp, out in zip(examples['instruction'], examples['input'], examples['output'])]
    return {"text": texts}

# Prepare dataset for finetuning
dataset = dataset.map(formatting_prompts_func, batched=True)

# Training setup with Huggingface's Trainer
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Batch size optimized for T4 GPU
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # Limited steps for quick results
        learning_rate=2e-4,
        fp16=True,  # Efficient for free T4
        logging_steps=1,
        output_dir="outputs"
    ),
)

# Show GPU memory stats before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Start training
trainer.train()

# Show memory and training time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"Training completed. Memory used = {used_memory - start_gpu_memory} GB.")

# Inference for code generation task
FastLanguageModel.for_inference(model)  # Enable faster inference
input_text = """
### Instruction:
Complete the Python function to calculate the factorial of a number.

### Input:
def factorial(n):

### Response:
"""
inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Saving the fine-tuned model (LoRA adapters)
model.save_pretrained("lora_model")  # Local save of LoRA adapters
tokenizer.save_pretrained("lora_model")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.879 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 13,631,488
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/

Step,Training Loss
1,1.555900
2,1.906400
3,1.868300
4,2.105000
5,1.511900
6,1.668300
7,1.420300
8,1.553600
9,1.566100
10,1.565600


Training completed. Memory used = 1.3050000000000006 GB.

### Instruction:
Complete the Python function to calculate the factorial of a number.

### Input:
def factorial(n):

### Response:
def factorial(n):
    """Return the factorial of n."""
    if n == 0 or n == 1:
        return 1
    else:
        return n * factorial(n - 1)



('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')